# Simulação das propriedades da exponencial

### Importar bibliotecas

In [1]:
import time
import numpy as np
import scipy.stats as st

## Probabilidade de uma variável aleatória exponencial ser menor do que outra
Sejam $X_{1}$ e $X_{2}$ variáveis aleatórias independentes com média ${mu}_{1} = 1/L_{1}$ e ${mu}_{2} = 1/L_{2}$<br>

* P[$X_{1} \leq X_{2}$] = $\frac{L_{1}}{L_{1}+L_{2}}$<br><br>

Podemos simular essa probabilidade com o seguinte algoritmo vetorial, implementado na função **pExpMenor**.<br>
* Sortear array ${exp}_{1}$ com $nSim$ valores exponenciais com média ${mu}_{1}$.<br>
* Sortear array ${exp}_{2}$ com $nSim$ valores exponenciais com média ${mu}_{2}$.<br>
* Calcular vetor $menor$ com valores $True$ se ${exp}_{1}$ < ${exp}_{2}$. <br>
* Retornar a quantidade de elementos no vetor $menor$ contendo valor True, dividida por $nSim$.<br><br>

In [2]:
def pExpMenor(MU1, MU2, nSim):
    exp1 = st.expon.rvs(scale=MU1, size=nSim)
    exp2 = st.expon.rvs(scale=MU2, size=nSim)
    menor = exp1 < exp2
    return np.count_nonzero(menor) / nSim

O código a seguir compara o valor de probabildade simulado pela função **pExpMenor** com o valor teórico.

In [3]:
mu1 = 1000
mu2 = 500
nSim = 10000

probT = (1/mu1)/(1/mu1+1/mu2)
t1 = time.perf_counter()
probS = pExpMenor(mu1, mu2, nSim)
t2 = time.perf_counter()
print('Probabilidade simulada:  {:.4f}'.format(probS))
print('Probabilidade teórica: {:.4f}'.format(probT))
print('Tempo de simulação: {:.4f}'.format(t2-t1))

Probabilidade simulada:  0.3360
Probabilidade teórica: 0.3333
Tempo de simulação: 0.0071


## Distribuição da soma de *N* variáveis aleatórias exponenciais
Sejam $X_{1}, X_{2}, \cdots , X_{N}$ variáveis aleatórias independentes com média $mu_{1} = 1/L$<br>
$X = X_{1} + X_{2} + \cdots + X_{N}$ tem distribuição Erlang com parâmetros *N* e *L*.
* $f_{X}(x)=\frac{L^{N}x^{N-1}e^{-Lx}}{\tau (N)}$
* $F_{X}(x)=1-\sum_{j=0}^{N-1}e^{-Lx}\frac{(Lx)^{j}}{j!}$

Com a biblioteca Scipy temos:<br>
* P[$X \leq x$] = st.gamma.cdf(x, a=N, scale=MU)<br><br>

## Algoritmo interativo
Podemos calcular simular a CDF da soma de $N$ variáveis aleatórias exponencias com média $MU$ com o seguinte algoritmo interativo:<br>
* Iniciar a variável $deuCerto$ com zero.<br>
* Sortear o array $EXP$ com $N$ variáveis aleatórias com média $MU$.<br>
* Calcular o array $soma$ contendo o somatório de $EXP$, ou seja, a soma de $N$ variáveis aleatórias exponenciais com média $MU$.<br>
* Incrementar $deuCerto$, e a soma for menor do que o valor $x$ para o qual queremos calcular a CDF (passado com argumento).<br>
* Retornar $deuCerto$ dividido por $nSim$.<br><br>

In [4]:
def somaExpCDFI(x, N, MU, nSim):
    deuCerto = 0
    for i in range(nSim):
        EXP = st.expon.rvs(scale=MU, size=N)
        soma = sum(EXP)
        if soma <= x:
            deuCerto = deuCerto + 1
    return deuCerto/nSim

O código a seguir compara o valor de probabildade simulado pela função **somaExpCDFI** com o valor da CDF da variável gamma calculado pela biblioteca Scipy.

In [5]:
x = 25
N = 10
MU = 3
nSim = 100000

probT = st.gamma.cdf(x, a=N, scale=MU)
t1 = time.perf_counter()
probS = somaExpCDFI(x, N, MU, nSim)
t2 = time.perf_counter()
print('Probabilidade simulada:  {:.4f}'.format(probS))
print('Probabilidade teórica: {:.4f}'.format(probT))
print('Tempo de simulação: {:.4f}'.format(t2-t1))

Probabilidade simulada:  0.3265
Probabilidade teórica: 0.3255
Tempo de simulação: 14.8270


## Algoritmo vetorial
Podemos calcular simular a CDF da soma de $N$ variáveis aleatórias exponencias com média $MU$ com o seguinte algoritmo vetorial implementado com a biblioteca numpy:<br>

Sortear a matriz $EXP$ com $nSim$ linhas e $N$ colunas.<br>
* Observação: Cada linha corresponde a uma simulação e contém os valores das $N$ exponencias sorteadas com média MU. <br>
* Dica: usar np.random.exponential(MU, [nSim, N]) <br><br>

Calcular o array $soma$ contendo o somatório de cada linha da matriz $EXP$, ou seja, cada elento tem a soma de $N$ variáveis aleatórias exponenciais com média $MU$.<br>
* Dica: usar np.sum(), observando que tem que somar as linhas. <br><br>

Calcular o array $menor$ que contém $True$ para cada linha da matriz cuja soma menor seja do que o valor $x$ para o qual queremos calcular a CDF (passado com argumento).<br>
* Dica: (soma <= x)<br><br>

Retornar a quantidade de elementos no vetor $menor$ contendo valor True, dividida por $nSim$.<br>
* Dica: usar a função np.count_nonzero para contar a quantidade de elementos com valor True.<br><br>

In [6]:
def somaExpCDFV(x, N, MU, nSim):
    # Gerar uma matriz nSim x N de valores exponenciais
    EXP = np.random.exponential(MU, [nSim, N])

    # Calcular a soma das variáveis exponenciais ao longo de cada linha
    soma = np.sum(EXP, axis=1)

    # Verificar onde a soma é menor ou igual a x
    menor = soma <= x

    # Retornar a proporção de casos onde a soma foi menor ou igual a x
    return np.count_nonzero(menor) / nSim


O código a seguir compara o valor de probabildade simulado pela função **somaExpCDFV** com o valor da CDF da variável gamma calculado pela biblioteca Scipy.

In [7]:
x = 25
N = 10
MU = 3
nSim = 100000

probT = st.gamma.cdf(x, a=N, scale=MU)
t1 = time.perf_counter()
probS = somaExpCDFV(x, N, MU, nSim)
t2 = time.perf_counter()
print('Probabilidade simulada:  {:.4f}'.format(probS))
print('Probabilidade teórica: {:.4f}'.format(probT))
print('Tempo de simulação: {:.4f}'.format(t2-t1))

Probabilidade simulada:  0.3245
Probabilidade teórica: 0.3255
Tempo de simulação: 0.0401


## Entrega
Completar o algoritmo.<br>
Imprimir para PDF.<br>
Enviar no AVA.<br>